
# Tipagem e Processamento de Dados

In [83]:
from ydata_profiling import ProfileReport 
import pandas as pd 
from unidecode import unidecode
import os

> **Nota:** Optei por utilizar apenas os dados de 2017 a 2024, pois a partir de 2017 passaram a ser incluídas informações de longitude e latitude, conforme indicado no dicionário disponível na pasta `reports`.


In [84]:
dirpath = os.path.join('..','..','data','raw','datatran','ocorrencia')
data  = [pd.read_csv(os.path.join(dirpath,file),encoding='latin1',sep=';') for file in os.listdir(dirpath)][10:]
data = pd.concat(data)

C:\Users\Carlos\AppData\Local\Temp\ipykernel_10532\88604282.py:2: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data  = [pd.read_csv(os.path.join(dirpath,file),encoding='latin1',sep=';') for file in os.listdir(dirpath)][10:]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_10532\88604282.py:2: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data  = [pd.read_csv(os.path.join(dirpath,file),encoding='latin1',sep=';') for file in os.listdir(dirpath)][10:]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_10532\88604282.py:2: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data  = [pd.read_csv(os.path.join(dirpath,file),encoding='latin1',sep=';') for file in os.listdir(dirpath)][10:]


In [85]:
# Testa tipos mixos
def is_mixed(col):
    return len(set(type(x) for x in col if pd.notna(x))) > 1

for c in data.columns:
    if is_mixed(data[c]):
        print(c)
# As colunas que tem mais de um tipo de dado são latitude e longitude, então vou transformar em string
data['latitude'] = data['latitude'].astype(str)
data['longitude'] = data['longitude'].astype(str)

latitude
longitude


In [86]:
# Alteração do nome data_inversa para data, e limpeza do nome das colunas
data.columns = data.columns.str.strip().str.lower()
data.rename(columns={'data_inversa':'data'},inplace=True)


In [87]:
# é possivel obter horario e dia da semana a partir da coluna data, então eliminei colunas horario e dia da semana
data['data'] = pd.to_datetime(data['data']+' '+data['horario'])
data.drop(['horario','dia_semana'],axis=1,inplace=True)


In [88]:
data['id'] = data['id'].astype(int)
data.set_index('id',inplace=True)

In [89]:
# Normalização de colunas categoricas que são strings
cat_upper = ['uf','municipio','delegacia','regional','uop']
for cat in cat_upper:
    data[cat] = data[cat].str.strip().str.upper()
    data[cat] = data[cat].apply(lambda s: unidecode(s) if isinstance(s,str) else s)
    data[cat] = data[cat].astype('category')
cat_lower = ['causa_acidente','tipo_acidente','classificacao_acidente','fase_dia','sentido_via','condicao_metereologica','tipo_pista','tracado_via','uso_solo']
for cat in cat_lower:
    data[cat] = data[cat].str.strip().str.lower()
    data[cat] = data[cat].apply(lambda s: unidecode(s) if isinstance(s,str) else s)
    data[cat] = data[cat].astype('category')



In [90]:
data['km'] = data['km'].str.replace(',','.').astype('float')
data['latitude'] = data['latitude'].str.strip().str.replace(',','.').astype(float)
data['longitude'] = data['longitude'].str.strip().str.replace(',','.').astype(float)
data['br'] = data['br'].astype('category')

In [91]:
data.to_pickle(os.path.join('..','..','data','processed','datatran_ocorrencias.pkl'))

In [92]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 560138 entries, 17 to 661598
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   data                    560138 non-null  datetime64[ns]
 1   uf                      560138 non-null  category      
 2   br                      560138 non-null  category      
 3   km                      560138 non-null  float64       
 4   municipio               560138 non-null  category      
 5   causa_acidente          560138 non-null  category      
 6   tipo_acidente           560097 non-null  category      
 7   classificacao_acidente  560129 non-null  category      
 8   fase_dia                560138 non-null  category      
 9   sentido_via             560138 non-null  category      
 10  condicao_metereologica  560138 non-null  category      
 11  tipo_pista              560138 non-null  category      
 12  tracado_via             560138 non